# Pandas

In [1]:
import pandas as pd
import numpy as np

In [2]:
# record script running time
from time import time
    
start_run_time = time()
time_taken = time() - start_run_time

# 1. Data Preparation

## 1.1. Create Demo DataFrame with 3 Data Type

1. Datetime
2. Numeric
3. Category

In [3]:
def random_df(seed,nrow):

    np.random.seed(seed)
    
    # datetime (freq = day)
    def random_date(start,end):
    
        min_date = pd.to_datetime(start)
        max_date = pd.to_datetime(end)
        
        d = (max_date - min_date).days + 1
        
        date = min_date + pd.to_timedelta(np.random.randint(d,size=nrow),unit='d')
    
        return date
    
    date_col = random_date(start='2018-01-01',end='2020-12-31')
    
    
    # numeric (with na)
    # following normal distribution having mean 'loc' and standard deviation 'scale'
    income = np.random.normal(loc=50000,scale=15000,size=nrow)
    gen_na = np.random.choice([1, 0], income.shape, p=[0.1, 0.9]).astype(bool)
    income[gen_na] = np.nan
    
    
    # random values between low and high.
    age = np.random.randint(low=1, high=100, size=nrow)
    
    # category
    insurance_type = np.random.choice([' OP ',' INP','TR','   '], nrow, replace=True,p=[0.4, 0.3, 0.2,0.1])
    
    _df_ = pd.DataFrame({'insurance_type'  : insurance_type,
                         'date' : date_col,
                         'age' : age,
                         'income' : income})
    
    return _df_

In [4]:
df = random_df(seed=123,nrow=50)

print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   insurance_type  50 non-null     object        
 1   date            50 non-null     datetime64[ns]
 2   age             50 non-null     int32         
 3   income          43 non-null     float64       
dtypes: datetime64[ns](1), float64(1), int32(1), object(1)
memory usage: 1.5+ KB
None


#### First 10 row of dataframe

In [5]:
df.head(10)

,insurance_type,date,age,income
0,OP,2020-11-07,98,NaN
1,INP,2018-04-17,40,33798.199209
2,TR,2018-04-07,83,44105.435919
3,INP,2018-04-24,93,59376.859539
4,,2019-10-01,39,24426.992678
5,OP,2018-02-17,6,45226.600720
6,INP,2018-03-15,10,42375.490333
7,,2020-07-31,41,61198.372674
8,OP,2018-08-13,69,30464.852872
9,OP,2018-04-22,88,51809.046069


## 1.2. Dictionary as DataFrame

In [6]:
Dict = { } 

i = 0

Dict[i],i = {'name': 'Cara', 'age': 25}, i+1
Dict[i],i = {'name': 'ABC', 'age': 40} , i+1

print(Dict) 

{0: {'name': 'Cara', 'age': 25}, 1: {'name': 'ABC', 'age': 40}}


In [7]:
Dict_df = pd.DataFrame(Dict).transpose()

Dict_df

,name,age
0,Cara,25
1,ABC,40


# 2. Data Cleaning
1. All Data Type: replace an empty string and records with only spaces
2. Object Data Type: trim string
3. Numeric Data Type: non-numeric convert to na

In [8]:
def Data_Cleaning(_df_):
    
    # Create copy
    df = _df_.copy()
    
    # 1. For all date type:
    # Replace an empty string and records with only spaces
    df.replace(r'^\s*$', np.nan, regex=True, inplace = True)
    
    # 2. Object data type
    obj_col = df.select_dtypes(['object']).columns
    print('*** Object coloumn include:',obj_col)
    # Trim string
    df[obj_col] = df[obj_col].apply(lambda x: x.str.strip())
    # Fill n.a. as "Missing"
    df[obj_col] = df[obj_col].fillna('Missing', inplace=False)
        
    # 3. Numeric data type
    num_col = df.select_dtypes(['number']).columns  
    print('*** Numeric coloumn include:',num_col)
    # Invalid value as np.nan')
    df[num_col] = df[num_col].apply(pd.to_numeric, errors='coerce')
    
    print('----- Data Cleaning Finish -----')
    
    return df

In [9]:
df = Data_Cleaning(df)

# checking function is work
print('Column value after data cleaning:')
print(df.insurance_type.value_counts())

*** Object coloumn include: Index(['insurance_type'], dtype='object')
*** Numeric coloumn include: Index(['age', 'income'], dtype='object')
----- Data Cleaning Finish -----
Column value after data cleaning:
OP         22
INP        20
TR          6
Missing     2
Name: insurance_type, dtype: int64


# 3. Data Processing

## 3.1 Data Selection

#### Select column(s) by data type

In [10]:
obj_col = df.select_dtypes(['object','category']).columns
num_col = df.select_dtypes(['number']).columns
date_col= df.select_dtypes(['datetime']).columns

print('obj_col:',obj_col)
print('num_col:',num_col)
print('date_col:',date_col)

obj_col: Index(['insurance_type'], dtype='object')
num_col: Index(['age', 'income'], dtype='object')
date_col: Index(['date'], dtype='object')


#### Select by row index and column name

In [11]:
df.iloc[0]['date']

Timestamp('2020-11-07 00:00:00')

#### Select by date range

In [12]:
start_date = '2019-01-16'
end_date = '2019-02-01'

mask = (df['date'] >= start_date) & (df['date'] <= end_date)

# subset_df: 1 py_id & filter date range
subset_df = df.loc[mask]

subset_df

,insurance_type,date,age,income
21,OP,2019-01-16,24,38231.313270
38,INP,2019-01-26,91,26556.112133


#### Select specified row/column pair (cell)

    Can assign value (e.g. df.at[20, 'age'] = 3)

In [13]:
df.at[20, 'date']

Timestamp('2018-04-10 00:00:00')

## 3.2 Data Manipulation

#### Sorting by Column Value

In [14]:
df.sort_values(['date','age'], axis=0, ascending=[True, True], inplace=True, ignore_index=True)

print(df.head())

  insurance_type       date  age        income
0            INP 2018-01-03   93  27808.592913
1             TR 2018-02-09   82  53057.428799
2             OP 2018-02-17    6  45226.600720
3             OP 2018-02-21   54           NaN
4            INP 2018-03-15   10  42375.490333


#### Rename Column

In [15]:
df_1 = df.copy()
df_1.rename(columns={'age': 'ageageage', 'date': 'datedate'}, inplace=True)

print(df_1.head())

  insurance_type   datedate  ageageage        income
0            INP 2018-01-03         93  27808.592913
1             TR 2018-02-09         82  53057.428799
2             OP 2018-02-17          6  45226.600720
3             OP 2018-02-21         54           NaN
4            INP 2018-03-15         10  42375.490333


#### Drop Column

In [16]:
df_2 = df.copy()
df_2.drop(['insurance_type', 'date'], axis=1, inplace=True)

print(df.head())

  insurance_type       date  age        income
0            INP 2018-01-03   93  27808.592913
1             TR 2018-02-09   82  53057.428799
2             OP 2018-02-17    6  45226.600720
3             OP 2018-02-21   54           NaN
4            INP 2018-03-15   10  42375.490333


#### Filter by Column Value

In [17]:
df_3 = df[df.age > 90]

print(df_3.head())
print('----------------------')
print(df_3.shape)

   insurance_type       date  age        income
0             INP 2018-01-03   93  27808.592913
10            INP 2018-04-24   93  59376.859539
19             TR 2019-01-07   99  40182.377797
21            INP 2019-01-26   91  26556.112133
38            INP 2020-04-06   91           NaN
----------------------
(6, 4)


#### Drop Duplicates

In [18]:
df_4 = df.copy()
df_4.drop_duplicates(subset=['insurance_type','age'], keep='last',inplace=True)

print(df_4.head())
print('----------------------')
print(df_4.shape)

  insurance_type       date  age        income
1             TR 2018-02-09   82  53057.428799
2             OP 2018-02-17    6  45226.600720
3             OP 2018-02-21   54           NaN
4            INP 2018-03-15   10  42375.490333
5             TR 2018-04-07   83  44105.435919
----------------------
(46, 4)


#### Value Counts

In [19]:
print(df['insurance_type'].value_counts(normalize=False,dropna=False))
print('----------------------')
print(df['insurance_type'].value_counts(normalize=True,dropna=False))

OP         22
INP        20
TR          6
Missing     2
Name: insurance_type, dtype: int64
----------------------
OP         0.44
INP        0.40
TR         0.12
Missing    0.04
Name: insurance_type, dtype: float64


#### Replace String in Column

In [20]:
df_5 = df.copy()

df_5['insurance_type'] = df_5['insurance_type'].astype(str).str.replace('P', 'Pat')

print(df_5['insurance_type'].value_counts(normalize=True,dropna=False))

OPat       0.44
INPat      0.40
TR         0.12
Missing    0.04
Name: insurance_type, dtype: float64


#### Create New Column Based on Another Column Value

In [21]:
df_6 = df.copy()

df_6['is_medical'] = np.where(df_6['insurance_type'].isin(['OP','INP']), '1', '0')

print(df_6.head())
print('----------------------')
print(df_6['is_medical'].value_counts(normalize=True,dropna=False))

  insurance_type       date  age        income is_medical
0            INP 2018-01-03   93  27808.592913          1
1             TR 2018-02-09   82  53057.428799          0
2             OP 2018-02-17    6  45226.600720          1
3             OP 2018-02-21   54           NaN          1
4            INP 2018-03-15   10  42375.490333          1
----------------------
1    0.84
0    0.16
Name: is_medical, dtype: float64


#### Return Index Based on Column Value

In [22]:
age_index_list = df.index[df['age'] > 80].tolist()
income_index_list = df.index[df['income'] > 45000].tolist()

print('----------------------')
print('age_index_list:')
print(age_index_list)

print('----------------------')
print('income_index_list:')
print(income_index_list)

----------------------
age_index_list:
[0, 1, 5, 9, 10, 16, 17, 19, 21, 23, 31, 37, 38, 40, 43, 47]
----------------------
income_index_list:
[1, 2, 6, 7, 9, 10, 12, 15, 16, 17, 18, 23, 24, 25, 26, 28, 29, 32, 33, 34, 37, 40, 41, 43, 44, 45, 46, 48]


#### Combine DataFrame (Append Column)

In [23]:
# column name of "left table"
left_col = ['a','b','c']
# column name of "right table"
right_col = ['aa','bb','cc']

# joined_df = pd.merge(df1, df2, how='left', left_on=left_col, right_on=right_col)

#### Append Dataframe (Append Row)

    Same Column Name: Append

    Different Column Name: New Column

In [24]:
# df = pd.concat([df1, df2],ignore_index=True)

# 4. Custom Function

#### Create New Column Based on Conditional Column Value

    Create Empty 2D List

    Assign Value in Empty 2D List by Append Method

    Replace String

In [25]:
list_2D = [[] for _ in range(len(df))]

age_index_list = df.index[df['age'] > 80].tolist()
income_index_list = df.index[df['income'] > 45000].tolist()

for i in age_index_list:
    list_2D[i].append('age more than 80')

for i in income_index_list:
    list_2D[i].append('income more than 45000')
    
print('First 5 element in list_2D:')
print(list_2D[0:5])
print('----------------------')
print('Assign list_2D as DF column:')
df['new_col'] = list_2D
# df['new_col'] = df['new_col'].astype(str).replace('[]', np.nan).str.replace('[', '').str.replace(']', '').str.replace("'", '')

df['new_col'] = df['new_col'].astype(str).replace('[]', np.nan).str.replace('[', '').str.replace(']', '').str.replace("'", '')


df.head(10)

First 5 element in list_2D:
[['age more than 80'], ['age more than 80', 'income more than 45000'], ['income more than 45000'], [], []]
----------------------
Assign list_2D as DF column:


,insurance_type,date,age,income,new_col
0,INP,2018-01-03,93,27808.592913,age more than 80
1,TR,2018-02-09,82,53057.428799,"age more than 80, income more than 45000"
2,OP,2018-02-17,6,45226.600720,income more than 45000
3,OP,2018-02-21,54,NaN,NaN
4,INP,2018-03-15,10,42375.490333,NaN
5,TR,2018-04-07,83,44105.435919,age more than 80
6,INP,2018-04-10,79,64164.758508,income more than 45000
7,INP,2018-04-17,16,64631.354824,income more than 45000
8,INP,2018-04-17,40,33798.199209,NaN
9,OP,2018-04-22,88,51809.046069,"age more than 80, income more than 45000"


#### Split DataFrame by Column

In [26]:
feature_list = ['insurance_type','age']

fea_df = df[feature_list]
non_fea_df = df.drop(feature_list, axis=1, inplace=False)

print('*** fea_df ***')
print(fea_df.head())
print('----------------------')
print('*** non_fea_df ***')
print(non_fea_df.head())

*** fea_df ***
  insurance_type  age
0            INP   93
1             TR   82
2             OP    6
3             OP   54
4            INP   10
----------------------
*** non_fea_df ***
        date        income                                   new_col
0 2018-01-03  27808.592913                          age more than 80
1 2018-02-09  53057.428799  age more than 80, income more than 45000
2 2018-02-17  45226.600720                    income more than 45000
3 2018-02-21           NaN                                       NaN
4 2018-03-15  42375.490333                                       NaN
